## This is my First Algorithm Trading mini project that will be made using pure OOPs knowledge and some external python modules such as Yahoo Finance aka yfinance and other battery included in python.

In [1]:
# !pip install yfinance -q

-q in above code denotes that the system quitely installs each and every file keeping the terminal clean and good looking.

In [2]:
import yfinance as yf
from datetime import datetime
import time

In [3]:
class TradingStrategy:
    def __init__(self, name):
        self.__name = name

    def generate_signal(self, price_data):
        return "Hold"
    
    @property
    def name(self):
        return self.__name

@property decorator is used to make private variables read only outside the class if wanted to access data.

In [4]:
class SMA_strategy(TradingStrategy):
    def __init__(self, short_window, long_window):
        super().__init__("Simple Moving Average Strategy")
        self.__short_window = short_window
        self.__long_window = long_window

    def generate_signal(self:object, price_data:list):
        if len(price_data) < self.__long_window:
            return "Hold"
        s_avg = sum(price_data[-self.__short_window:]) / self.__short_window
        l_avg = sum(price_data[-self.__long_window:]) / self.__long_window

        if s_avg > l_avg:
            return "Buy"
        elif s_avg < l_avg:
            return "Sell"
        else:
            return "Hold"
        
    @property
    def short_window(self):
        return self.__short_window
    
    @property
    def long_window(self):
        return self.__long_window


In [5]:
class Trade:
    def __init__(self, strategy_name, signal, amount):
        self.__sname = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timeStamp = datetime.now()

    def execute(self):
        print(f"Executing trade for {self.__sname} having signal {self.__signal} using {self.__amount} at {self.__timeStamp}")

    @property
    def strategy_name(self):
        return self.__sname
    
    @property
    def signal(self):
        return self.__signal
    
    @property
    def amount(self):
        return self.__amount
    
    @property
    def timeStamp(self):
        return self.__timeStamp

In [6]:
class MockTradingAPI:
    def __init__(self, balance):
        self.__balance = balance

    def place_order(self, trade, price):
        total = trade.amount * price
        if trade.signal == "Buy" and self.__balance >= total:
            self.__balance -= total
            print(f"Executed {trade.signal} for amount {total}, remaining balance is {self.__balance}")
        
        elif trade.signal == "Sell":
            self.__balance += total
            print(f"Executed {trade.signal} for amount {total}, remaining balance is {self.__balance}")

        else:
            print("Invalid signal or not sufficient balance in account")
    
    @property
    def balance(self):
        return self.__balance

In [7]:
class TradingSystem:
    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []

    def fetch_market_data(self):
        data = yf.download(self.__symbol, period="1d", interval="1m")
        if not data.empty:
            price = data["Close"].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data) > self.__strategy.long_window:
                self.__price_data.pop(0)
            print(f"Fetched new price data {price}")
        else:
            print(f"data not fetched for {self.__symbol}")

    def run(self):
        self.fetch_market_data()
        signal = self.__strategy.generate_signal(self.__price_data)
        print(f"Generated Signal {signal}")

        if signal in ["Buy", "Sell"] and len(self.__price_data) > 0:
            trade = Trade(self.__strategy.name, signal, 1)
            trade.execute()
            self.__api.place_order(trade, self.__price_data[-1])
    
    @property
    def api(self):
        return self.__api
    
    @property
    def strategy(self):
        return self.__strategy
    
    @property
    def symbol(self):
        return self.__symbol
    
    @property
    def price_data(self):
        return self.__price_data

In [ ]:
if __name__ == "__main__":
    symbol = "AAPL"

    api = MockTradingAPI(10000)
    strategy = SMA_strategy(short_window=3, long_window=5)
    system = TradingSystem(api, strategy, symbol)

    for _ in range(10):
        system.run()
        print(f"Remaining balance: {api.balance}")
        time.sleep(60)